In [1]:
! pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from rank_bm25 import BM25Okapi

In [4]:
import pandas as pd

df=pd.read_csv('/content/CompleteMainText.csv')
df_just_text=df[['DocID','Text','URL']]
df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")
corpus = df_just_text['Text'].tolist()
print(len(corpus))

198


<ipython-input-4-b2a201fa1d6d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")


In [5]:
# df_corpus_token['Text'] = [string.replace("\n"," ") for string in df_just_text['Text']]
# print(df_corpus_token['Text'][0])

In [6]:
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [7]:
df_tokenized_corpus=df_just_text.copy()
df_tokenized_corpus['Text'] = [doc.split(" ") for doc in df_just_text['Text']]
tokenized_corpus = df_tokenized_corpus['Text'].tolist()
# df_tokenized_corpus['Text'][0]
# df_tokenized_corpus['Text'][0]

In [8]:
bm25 = BM25Okapi(tokenized_corpus)

In [9]:
# df_corpus=df_just_text.copy()
# df_corpus_token=df_just_text.copy()
# df_corpus_token['Text'] = [doc.replace("\n"," ").split(" ") for doc in df_corpus['Text']]

In [10]:
# print(df_corpus_token)

In [11]:
# bm25 = BM25Okapi(df_corpus_token['Text'])

In [12]:
query_list = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

tokenized_query=[]
for query in query_list:
  tokenized_query.append(query.split(" "))

In [ ]:
# tokenized_query

In [ ]:
# bm25.get_top_n(tokenized_query[0], corpus, n=5)

In [ ]:
# doc_scores = bm25.get_scores(tokenized_query)

In [ ]:
# top_text=bm25.get_top_n(tokenized_query, corpus, n=2)

In [ ]:
# doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]

In [13]:

final_document_List=[]
diction={}

for a,single_query in enumerate(tokenized_query):
  top_text=bm25.get_top_n(single_query, corpus, n=10)
  # print(top_text)
  # print(df_corpus_token['Text']==top_text[0])
  # mask = df_corpus_token['Title'].apply(lambda x: set(top_text[0]).issubset(set(x)) if isinstance(x, list) else False)
  # doc_Index = df_corpus_token[mask]['DocID']
  # print(doc_Index)

  if query_list[a] not in diction:
        diction[query_list[a]] = []
  
  doc_scores = bm25.get_scores(single_query)
  final_score=sorted(doc_scores)
  for i,s in enumerate(top_text):
    # print(s)
    # print(df_corpus[df_corpus_token['Text'] == s, 'DocID'].iloc[0])
    index=corpus.index(s)
    # print("IndexValuue is->"+ str(index))
    # print("DFID is ->"+str(df_just_text.iloc[index]['DocID']))
    # index = df_just_text.loc[df_just_text['Text']==s].index.tolist()[0]
    # index = df_just_text.loc[df_just_text['Text']==s]['DocID'].item()

    # print(index)
    doc_index=df_just_text.iloc[index]['DocID']
    url=df_just_text.iloc[index]['URL']
    # if doc_index>20:
    #   doc_index+=2
    # elif doc_index<20 and doc_index>8:
    #   doc_index+=1

    # doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]
    # doc_index=df.loc[df['Title'] == title, 'DocID'].iloc[0]
    title=df.iloc[index]['Title']
    # print("Document ID-> "+str(index))
    # print("Title ->"+str(title))
    score=final_score[-(i+1)]
    diction[query_list[a]].append((doc_index,title,score,url))
    # print("Rank"+str(i)+"DocID-> :"+str(index)

for query in diction:
  print("QUERY-> "+query)
  for tup in diction[query]:
    print(tup)

QUERY-> fidgety child unable to sit still
(116, "How to Understand Your Child's Temperament - HealthyChildren.org", 9.828414820361512, 'https://healthychildren.org/English/ages-stages/gradeschool/Pages/How-to-Understand-Your-Childs-Temperament.aspx')
(180, "Some Kids Really Are More Difficult—Here's Why It Happens and How to Deal", 9.50575433208952, 'https://www.parents.com/toddlers-preschoolers/discipline/some-kids-really-are-more-difficult-heres-why-it-happens-and-how-to-deal/')
(25, 'Common Time-Out Mistakes and How to Solve Them', 8.977202449465551, 'https://www.parents.com/toddlers-preschoolers/discipline/time-out/common-time-out-mistakes-and-how-to-solve-them/')
(49, 'Toilet Training  (for Parents) - Nemours KidsHealth', 8.58987526267408, 'https://kidshealth.org/en/parents/toilet-teaching.html#catbehavior')
(112, 'Developmental Milestones: 4 to 5 Year Olds - HealthyChildren.org', 8.417528536518333, 'https://healthychildren.org/English/ages-stages/preschool/Pages/Developmental-Mil

After Getting top 10 results from BM25, we will feed it into semantic search

In [14]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [15]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [16]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [17]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [19]:
import pickle
embeddings_dataset = pickle.load(open('/content/multi-qa-mpnet-base-dot-v1_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 200
})

In [20]:
df_embeddings = embeddings_dataset.to_pandas()
df_embeddings

,Title,DocID,URL,Embeddings
0,Child Development | Psychology Today,0,https://www.psychologytoday.com/us/basics/chil...,"[-0.23532958, 0.0651451, -0.25807574, 0.015225..."
1,Parenting a Child with ADHD | Psychology Today,1,https://www.psychologytoday.com/us/basics/adhd...,"[0.1437849, -0.25766706, -0.23332703, -0.07706..."
2,Raising an Adopted Child | Psychology Today,2,https://www.psychologytoday.com/us/basics/adop...,"[-0.047560602, -0.14377627, -0.18156454, -0.08..."
3,Parenting a Child with Autism | Psychology Today,3,https://www.psychologytoday.com/us/basics/auti...,"[0.038312867, -0.32781175, -0.191678, 0.001177..."
4,Ending Homework Battles | Psychology Today,4,https://www.psychologytoday.com/us/blog/how-ra...,"[0.40369615, -0.30819276, -0.15987273, -0.1149..."
...,...,...,...,...
195,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.parents.com/parenting/better-paren...,"[0.23376556, -0.13408136, -0.31302536, -0.1441..."
196,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.parents.com/recipes/scoop-on-food/...,"[0.01389085, -0.13097522, -0.32682866, -0.2900..."
197,Separation and Social Anxiety in Kids,197,https://www.parents.com/toddlers-preschoolers/...,"[-0.013510818, -0.30817795, -0.2162156, -0.065..."
198,How Much Does My Kid Need to Eat?,198,https://www.parents.com/toddlers-preschoolers/...,"[-0.09462826, 0.16608414, -0.28048867, -0.0260..."


Final Hybrid Search

In [ ]:
# import pickle
# embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))
# embeddings_dataset
import pandas as pd

result_dict={}
final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])


question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]


from datasets import Dataset

# assume df is a pandas dataframe


for i,query in enumerate(question):
  temp_df_embeddings = pd.DataFrame(columns=['Title', 'DocID', 'URL', 'Embeddings', 'Text'])
  
  for individual_doc in diction[query]:
    DocID = individual_doc[0]
    temp_df_embeddings = temp_df_embeddings.append(df_embeddings[df_embeddings['DocID'] == DocID], ignore_index=True)
  # print(temp_df_embeddings)
  embeddings_dataset = Dataset.from_pandas(temp_df_embeddings)
  embeddings_dataset.add_faiss_index(column="Embeddings")

  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  
  
  print("------------------------")
  print("QUERY-> "+str(i+1)+" "+question[i])
  print("------------------------")

  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title = row.Title
    docID= row.DocID
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    print(f"DocID: {dID}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
  


# for a,single_query in enumerate(tokenized_query):
  # top_text=bm25.get_top_n(single_query, corpus, n=10)
  # embeddings_list = []



  # embeddings_dataset=[[get_embeddings([top_text[i]]).cpu().detach().numpy() for i in range(10)]]
  # for result_string in top_text:

    #get the docID for each top_text from the df_just_text dataframe
    # print(df_embeddings[df_embeddings['Text'] == result_string])
    # print(df_embeddings[df_embeddings['Text'].str.contains(result_string)])

    # temp_df_embeddings.append(df_embeddings[df_embeddings['Text'] == result_string], ignore_index=True)
    
    # doc_index=df_just_text.iloc[index]['DocID']
  # print(temp_df_embeddings)
  # for string_result in top_text:
  # for i,question_embedding in enumerate(question):
  #   temp_df_embeddings = pd.DataFrame(columns=['Title', 'DocID', 'URL', 'Embeddings', 'Text'])
  #   question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  #   scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  #   samples_df = pd.DataFrame.from_dict(samples)
  #   samples_df["scores"] = scores
  #   samples_df.sort_values("scores", ascending=False, inplace=True)
  #   samples_df["scores"] = scores
  #   samples_df.sort_values("scores", ascending=False, inplace=True)
  #   # print("------------------------")
  #   # print("QUERY-> "+str(i)+" "+question[i])
  #   # print("------------------------")
    
  #   query=question[i]
  #   rank=1
  #   for _, row in samples_df.iterrows():
  #     title = row.Title
  #     docID= df_just_text.iloc[index]['DocID']
  #     qID="q_"+str(i+1)
  #     dID="d_"+str(docID)
  #     # print(f"TITLE: {row.Title}")
  #     # print(f"SCORE: {row.scores}")
  #     # print(f"DocID: {dID}")
  #     # print(f"URL: {row.URL}")
  #     # print("=" * 50)
  #     # print()
  #     if qID not in result_dict:
  #       result_dict[qID]={}
  #       result_dict[qID][dID]=row.scores
  #     else:
  #       result_dict[qID][dID]=row.scores
  #     final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

  #     rank+=1

In [22]:
print(result_dict)

{'q_1': {'d_149': 45.98992156982422, 'd_25': 45.94900131225586, 'd_180': 37.002220153808594, 'd_116': 35.48847961425781, 'd_112': 35.326175689697266}, 'q_2': {'d_167': 46.45101547241211, 'd_116': 44.73643493652344, 'd_68': 40.67823028564453, 'd_175': 39.69631576538086, 'd_52': 39.41313934326172}, 'q_3': {'d_6': 34.702239990234375, 'd_80': 29.018587112426758, 'd_113': 29.018587112426758, 'd_1': 25.63574981689453}, 'q_4': {'d_188': 29.912511825561523, 'd_165': 26.64927864074707, 'd_189': 25.07183265686035, 'd_191': 23.96661376953125, 'd_157': 19.50908851623535}, 'q_5': {'d_186': 42.14105987548828, 'd_74': 28.604698181152344, 'd_161': 28.604698181152344, 'd_12': 27.782909393310547}, 'q_6': {'d_125': 32.20671844482422, 'd_53': 28.746522903442383, 'd_90': 26.14396858215332, 'd_193': 24.963268280029297, 'd_50': 24.415050506591797}, 'q_7': {'d_48': 39.12376022338867, 'd_193': 38.37458419799805, 'd_4': 36.9064826965332, 'd_157': 34.50048828125, 'd_166': 29.968881607055664}, 'q_8': {'d_1': 35.8

In [23]:
!pip install ranx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautocorpus: filename=pyautocorpus-0.1.9-cp310-cp310-linux_x86_64.whl size=189816 sha256=47cd3b4b6823cda85c2e7441d21851745d74fa7cb1a6bc1ad81f8e566449c5bb
  Stored in directory: /root/.cache/pip/wheels/e2/be/43/ec1e16468c393f6e474e3be

In [24]:
from ranx import Qrels, Run
from ranx import evaluate

In [25]:
qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }


run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [26]:

final_result=evaluate(qrels, run, ["map", "ndcg"])


print(final_result)

{'map': 0.12675438596491226, 'ndcg': 0.25403182528152296}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the methods and results
methods = [
    "Semantic_Search - multi-qa-mpnet-base-dot-v1",
    "Semantic Search - all-MiniLM-L12-v2",
    "Semantic_Search - all-mpnet-base-v2",
    "Hybrid: BM25 + multi-qa-mpnet-base-dot-v1",
    "Semantic_Search - all-distilroberta-v1",
    "BM25",
    "Semantic_Search - distilbert-base-nli-stsb-mean-tokens"
]
map_results = [0.145344, 0.1316218, 0.12928, 0.126754, 0.11788, 0.10778, 0.07738]
ndcg_results = [0.2850852, 0.262336, 0.25413, 0.254031, 0.23726, 0.23948, 0.14938]

# Define the figure size
fig, ax = plt.subplots(figsize=(10, 5))

# Set the bar width and spacing
bar_width = 0.35
bar_spacing = 0.1

# Define the pastel colors
map_color = '#809bce'
ndcg_color = '#95b8d1'

# Create the x-axis values for the bars
x = np.arange(len(methods))

# Plot the MAP and NDCG bars side by side
map_bars = ax.bar(x - bar_width/2 - bar_spacing/2, map_results, bar_width, label="MAP", color=map_color)
ndcg_bars = ax.bar(x + bar_width/2 + bar_spacing/2, ndcg_results, bar_width, label="NDCG", color=ndcg_color)

# Add the method labels to the x-axis
ax.set_xticks(x)
ax.set_xticklabels(methods, rotation=90)

# Add axis labels and a title
ax.set_ylabel("Score")
ax.set_title("Comparison of Search Results")

# Add a legend
ax.legend()

# Display the plot
plt.show()
